Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
#rename latitude and longitude columns in citybike dataframe
vancouver_stations_df.rename(columns={"latitude": "Latitude", "longitude": "Longitude"}, inplace=True)

#join yelp and citybike dataframes
citybike_yelp_df = pd.merge(vancouver_stations_df, yelp_places_df, on=["Latitude", "Longitude"], how="outer")
citybike_yelp_df

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#distribution of ratings 

plt.figure(figsize=(10, 6))
sns.histplot(citybike_yelp_df['Rating'], bins=20, kde=True, color='blue')
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequency')
Heatmap of Bike Stations and Nearby Placesplt.show()

In [ ]:
# heatmap of bike stations and nearby places
import folium
from folium.plugins import HeatMap

# Initialize a map
city_map = folium.Map(location=[49.2827, -123.1207], zoom_start=13)

# Add bike station and POI locations to the heatmap
heat_data = [[Latitude, Longitude] for Latitude, Longitude in zip(citybike_yelp_df['Latitude'], citybike_yelp_df['Longitude'])]
HeatMap(heat_data).add_to(city_map)

# Show map
city_map


In [ ]:
#Clustered Map with Bike Stations and POIs

from folium.plugins import MarkerCluster

m = folium.Map(location=[49.2827, -123.1207], zoom_start=13)
marker_cluster = MarkerCluster().add_to(m)

for idx, row in citybike_yelp_df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Category']).add_to(marker_cluster)

m


In [ ]:
#histogram of available bikes

sns.histplot(citybike_yelp_df['free_bikes'], kde=True, bins=20)

In [ ]:
#Heatmap of Latitude and Longitude

plt.figure(figsize=(12, 8))
sns.scatterplot(x='Longitude', y='Latitude', data=citybike_yelp_df, hue='Category', palette='viridis', s=100, alpha=0.7, legend=False)
plt.title('Map of Places (Latitude vs Longitude)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

#plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
#It's difficult to analyze categories, because one cell may contain multiple categories, so i will leave only the first one

citybike_yelp_df['Category'] = citybike_yelp_df['Category'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

# Plotting the scatter plot with only the first category
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Longitude', y='Latitude', data=citybike_yelp_df, hue='Category', palette='viridis', s=100, alpha=0.7)
plt.xlabel('')
plt.ylabel('')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3
import pandas as pd

# create SQLite database using data from Yelp and CityBike

# 1. Connect to the SQLite database (it will create a new database file if it doesn't exist)
conn = sqlite3.connect('citybike_yelp_data.db')  
cursor = conn.cursor()

# 2. Create a table to store the data 
cursor.execute('''
CREATE TABLE IF NOT EXISTS bike_yelp_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Latitude REAL,
    Longitude REAL,
    free_bikes INTEGER,
    empty_slots INTEGER,
    slots INTEGER,
    uid INTEGER,
    Name TEXT,
    Category TEXT,
    Rating REAL,
    Review_Count INTEGER,
    Address TEXT,
    Phone TEXT,
    Website TEXT,
    Price Range TEXT,
    Photos TEXT
)
''')

# 3. Convert DataFrame to SQL table
citybike_yelp_df.to_sql('bike_yelp_data', conn, if_exists='replace', index=False)

# 4. Commit the transaction and close the connection
conn.commit()
conn.close()